## Imports

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy import units as u

## Load catalogs

In [ ]:
gaia_df = pd.read_csv('Gaia-Stripe82/Gaia_Stripe82_All.csv', dtype=str)
gaia_coods = gaia_df[['ra', 'dec']].to_numpy(np.float64)*u.degree
gaia_cat = SkyCoord(gaia_coods, frame='icrs')

sdss_df = pd.read_csv("SDSS-Stripe82/SDSS_Stripe82_All.csv", dtype=str)
sdss_coods = sdss_df[['ra', 'dec']].to_numpy(np.float64)*u.degree
sdss_cat = SkyCoord(sdss_coods, frame='icrs')

## Crossmatch and Output csv

In [ ]:
def get_partition_name (name, arc1, arc2) :
    return "{}_{}".format(name,
        "gt-{}".format(arc2) if arc1 is None else (
            "le-{}".format(arc1) if arc2 is None else "{}-b-{}".format(arc1, arc2)
        )
    )

def get_match_inds(matches, arc1, arc2) :
    assert arc1 is not None or arc2 is not None
    tol1 = None if arc1 is None else arc1/3600
    tol2 = None if arc2 is None else arc2/3600
    
    bools = matches > tol2 if tol1 is None else (
        np.logical_and(matches <= tol1, matches != 0) if tol2 is None
        else np.logical_and(matches > tol1, matches <= tol2)
    )
    
    return np.argwhere(bools).flatten()

def partition_zip (lo, hi, step) :
    return zip(
        [lo] + list(range(lo, hi, step)) + [None], 
        [None] + list(range(lo + step, hi + step, step)) + [hi]
    )
    
base = "SDSSxGaia"

lo_cross = 10
hi_cross = 100
step_cross = 10
lo_self = 1
hi_self = 10
step_self = 1
do_write = False

for arc1, arc2 in partition_zip(lo_cross, hi_cross, step_cross) :
    matches = np.array([
        sdss_cat.match_to_catalog_sky(gaia_cat, 1)[1]
    ]).flatten()
    
    matched_ids = get_match_inds(matches, arc1, arc2)
    cleaned_df = sdss_df.loc[matched_ids].reset_index()
    
    cfold = os.path.join(base, get_partition_name("Matches", arc1, arc2))
    if not os.path.isdir(cfold) : os.mkdir(cfold)
        
    if do_write : cleaned_df.to_csv(os.path.join(cfold, 'coods.csv'), index=False)
    cleaned_coods = cleaned_df[['ra', 'dec']].to_numpy(np.float64)*u.degree
    cleaned_cat = SkyCoord(cleaned_coods)
    cleaned_matches = np.array([
        cleaned_cat.match_to_catalog_sky(cleaned_cat, 2)[1]
    ]).flatten()

    glist = []
    for a1, a2 in partition_zip(lo_self, hi_self, step_self) :
        inds = get_match_inds(cleaned_matches, a1, a2)
        fname = get_partition_name("coods", a1, a2) + ".csv"
        fpath = os.path.join(base, cfold, fname)
        glist += [len(inds)]
        if do_write : cleaned_df.iloc[inds][cleaned_df.columns[1:]].to_csv(fpath, index=False)
    
    print(arc1, arc2)
    
    fig, ax = plt.subplots(1, 1)
    fig.set_figheight(5)
    fig.set_figwidth(7.5)
    ax.bar(range(0, hi_self+step_self, step_self), glist, 0.8, align='edge')
    ax.set_xticks(range(0, hi_self + step_self, step_self))
    ax.set_xlabel('Matching Distance', fontsize=15)
    ax.set_ylabel('Match count', fontsize=15)
    fig.savefig(os.path.join(cfold, "match_bar.png"))
    plt.close()

## Cross-Matching Distribution Data

In [ ]:
parts = ! wc -l SDSSxGaia/Matches_le-10/coods.csv SDSSxGaia/Matches*/coods.csv SDSSxGaia/Matches_gt-100/coods.csv \
    | grep -o "[0-9]*[0-9] " | head -11 | tr -d "[:blank:]"
parts = list(map(lambda x : int(x) - 1, parts))

## Plot Distribution

In [ ]:
fig, ax = plt.subplots(1,1)
fig.set_figheight(5)
fig.set_figwidth(7.5)
ax.bar(range(0, hi_cross + step_cross, step_cross), parts, 0.8*10, align='edge')
ax.set_xticks(range(0, hi_cross + step_cross, step_cross))
ax.set_xlabel('Matching Distance', fontsize=15)
ax.set_ylabel('Match count', fontsize=15)


fig.savefig(os.path.join(base, 'SDSSxGaia_match_bar.png'))
pass